In [1]:
import mne, glob
import mne.io
from scipy.io import loadmat
import h5py
import pandas as pd
import pymatreader
import timeit
import numpy as np
import matplotlib.pyplot as plt
#mne.sys_info()
from copy import deepcopy
import os
# import our functions
from mne_methods import *
from mne.preprocessing import (ICA, corrmap, create_ecg_epochs, 
                               create_eog_epochs)

## Data sets

In [2]:
sim_break_test_data = [[r'EEG_manual_100-25cha_0noise_1000', 10, [[] for x in range(6)], [], ["Na"]],
                       [r'EEG_manual_100-25cha_1noise_1000', 10, [['ELA'] for x in range(6)], ['ELA'], ["Na"]],
                       [r'EEG_manual_100-25cha_2noise_1000', 10, [['ELA','ELB'] for x in range(6)], ['ELA','ELB'], ["Na"]],
                       [r'EEG_manual_100-25cha_3noise_1000', 10, [['ELA','ELB','ELC'] for x in range(6)], ['ELA','ELB','ELC'], ["Na"]],
                       [r'EEG_manual_100-25cha_4noise_1000', 10, [['ELA','ELB','ELC','ELT'] for x in range(6)], ['ELA','ELB','ELC','ELT'], ["Na"]],
                       [r'EEG_manual_100-25cha_5noise_1000', 10, [['ELA','ELB','ELC','ELT','ELE'] for x in range(6)], ['ELA','ELB','ELC','ELT','ELE'], ["Na"]],
                       [r'EEG_manual_100-25cha_6noise_1000', 10, [['ELA','ELB','ELC','ELT','ELE','ELI'] for x in range(6)], ['ELA','ELB','ELC','ELT','ELE','ELI'], ["Na"]],
                       [r'EEG_manual_100-25cha_7noise_1000', 10, [['ELA','ELB','ELC','ELT','ELE','ELI','ERA'] for x in range(6)], ['ELA','ELB','ELC','ELT','ELE','ELI','ERA'], ["Na"]],
                       [r'EEG_manual_100-25cha_8noise_1000', 10, [['ELA','ELB','ELC','ELT','ELE','ELI','ERA','ERB'] for x in range(6)], ['ELA','ELB','ELC','ELT','ELE','ELI','ERA','ERB'], ["Na"]],
                       [r'EEG_manual_100-25cha_9noise_1000', 10, [['ELA','ELB','ELC','ELT','ELE','ELI','ERA','ERB','ERC'] for x in range(6)], ['ELA','ELB','ELC','ELT','ELE','ELI','ERA','ERB','ERC'], ["Na"]],
                       [r'EEG_manual_100-25cha_10noise_1000', 10, [['ELA','ELB','ELC','ELT','ELE','ELI','ERA','ERB','ERC','ERT'] for x in range(6)], ['ELA','ELB','ELC','ELT','ELE','ELI','ERA','ERB','ERC','ERT'], ["Na"]],
                       [r'EEG_manual_100-25cha_11noise_1000', 10, [['ELA','ELB','ELC','ELT','ELE','ELI','ERA','ERB','ERC','ERT','ERE'] for x in range(6)], ['ELA','ELB','ELC','ELT','ELE','ELI','ERA','ERB','ERC','ERT','ERE'], ["Na"]],
                       [r'EEG_manual_100-25cha_12noise_1000', 10, [['ELA','ELB','ELC','ELT','ELE','ELI','ERA','ERB','ERC','ERT','ERE','ERI'] for x in range(6)], ['ELA','ELB','ELC','ELT','ELE','ELI','ERA','ERB','ERC','ERT','ERE','ERI'], ["Na"]],
                       [r'EEG_manual_100-25cha_13noise_1000', 10, [['ELA','ELB','ELC','ELT','ELE','ELI','ERA','ERB','ERC','ERT','ERE','ERI','EOGr'] for x in range(6)], ['ELA','ELB','ELC','ELT','ELE','ELI','ERA','ERB','ERC','ERT','ERE','ERI','EOGr'], ["Na"]],
                       [r'EEG_manual_100-25cha_14noise_1000', 10, [['ELA','ELB','ELC','ELT','ELE','ELI','ERA','ERB','ERC','ERT','ERE','ERI','EOGr','EOGl'] for x in range(6)], ['ELA','ELB','ELC','ELT','ELE','ELI','ERA','ERB','ERC','ERT','ERE','ERI','EOGr','EOGl'], ["Na"]],
                       [r'EEG_manual_100-25cha_15noise_1000', 10, [['ELA','ELB','ELC','ELT','ELE','ELI','ERA','ERB','ERC','ERT','ERE','ERI','EOGr','EOGl','EMGl'] for x in range(6)], ['ELA','ELB','ELC','ELT','ELE','ELI','ERA','ERB','ERC','ERT','ERE','ERI','EOGr','EOGl','EMGl'], ["Na"]]]

In [3]:
print(len(sim_break_test_data))

16


## Helper functions

In [4]:
def check_file_exists(data_set, simulation):
    if simulation:
        local_folder = r"D:\Bachelor-Project-Data\Manual_data_28_05_2023"
    else:
        local_folder = r"D:\Bachelor-Project-Data\Kaare\study_1A_mat_simple"
    
    for data in data_set:
        folder_path = local_folder + "\ "[:-1] + data[0]
        
        if not os.path.isfile(folder_path + ".mat"):
            print(folder_path)

In [5]:
check_file_exists(sim_break_test_data, True)

In [6]:
def count_types(data_set):
    catogory_dictonary = {}
    
    # Runs through each data point in the data set
    for data_point in range(len(data_set)):
        
        #Runs through each category type:
        for category_type in data_set[data_point][4]:
            if category_type in catogory_dictonary.keys():
                # Add 1 to the type
                catogory_dictonary.update({category_type: catogory_dictonary.get(category_type) + 1})
            else:
                # Add a new type to the dictonary
                catogory_dictonary.update({category_type: 1})
    return catogory_dictonary

In [7]:
count_types(sim_break_test_data)

{'Na': 16}

## Optimisation Broken Electrode classifier

In [8]:
def optimise_set(data_set, simulation, parameters):
    # Empty list to the output.
    outputs = [] 
    
    start_cont = timeit.default_timer() # starts timer
    for data in data_set:
        #Running through every data point
        start = timeit.default_timer() # starts timer
        
        # Extracting the information of the data.
        if simulation:
            local_folder = r"D:\Bachelor-Project-Data\Manual_data_28_05_2023"
        else:
            local_folder = r"D:\Bachelor-Project-Data\Kaare\study_1A_mat_simple"
        folder_path = local_folder + "\ "[:-1] + data[0] #S_17/night_1\EEG_raw
        time_start = data[1]
        artifact_labels = data[2]
        broken_labels = data[3]
        
        # Loading the full data.
        raw_all = load_mne.file_to_raw(folder_path + ".mat")
        
        # Cropping it to the 60 seconds + buffer interval.
        raw_all = raw_all.crop(tmin=time_start - 4, tmax = time_start + 64)
        
        if simulation:
            #No preprocess
            raw_all_processed = raw_all.crop(tmin=4, tmax=64)

        else:
            # Preprocess
            raw_all_processed, NaN = load_mne.preprocess(raw_all)
            # Remove bufer interval.
            raw_all_processed = raw_all_processed.crop(tmin=4, tmax=64)
        
        # Prepares to loop through intervals.
        intv_count = 6
        intv_size = 10
        intv_end = intv_count * intv_size + 1 #Etc. 6*10 + 1
        time_intv = [x for x in range(0, intv_end, intv_size)]
        
        # ICA on intervals
        for intv_idx in range(intv_count):
            # Running through each interval
            folder_int = folder_path + "_time=" + str(time_start + time_intv[intv_idx]) + "-" + str(time_start +time_intv[intv_idx + 1])
            
            # Cropping to the interval to perform multiple functions [0, 10], [10, 20] etc.
            raw_crop_int = raw_all_processed.copy().crop(tmin = time_intv[intv_idx], tmax = time_intv[intv_idx + 1])
            
            # Runs multiple ICA for 2 - 15 number of components, both measures and matrices are used.
            ICA_subfunctions.multiple_ica(raw_crop_int, 2, 15, folder_int, matrix=parameters[1], cross_measure=parameters[0])
            
            # Runs multi_plot to get a summary of all 2 - 15 components.
            if type(parameters[1]) is not list and type(parameters[0]) is not list:
                ICA_subfunctions.multi_plot(2, 15, folder_int, matrix_list=[parameters[1]], cross_measure=[parameters[0]])
            else:
                ICA_subfunctions.multi_plot(2, 15, folder_int, matrix_list=parameters[1], cross_measure=parameters[0])
        
        if type(parameters[3]) is not list and type(parameters[2]) is not list and type(parameters[1]) is not list and type(parameters[0]) is not list:
            # Running the artifact classifier with relative procent on each ICA interval
            artifact_intervals = []
            for intv_idx in range(intv_count):
                folder_int = folder_path + "_time=" + str(time_start + time_intv[intv_idx]) + "-" + str(time_start +time_intv[intv_idx + 1])
                artifact_intervals.append(ICA_subfunctions.artifact_classifier(folder_int, parameters[2], cross=parameters[0], matrix=parameters[1]))

            # Running the broken electrode classifier with number of 10 s. intervals in row.
            broken_pred = ICA_subfunctions.broken_channel_classifier(artifact_intervals, broken_interval_size = parameters[3])

            # Output the predictions for the combination of parameters.
            outputs.append([data[0], time_start, parameters[0], parameters[1], parameters[2], parameters[3], artifact_intervals, broken_pred])
        else:
            # Artifact classifier
            for art_int in parameters[3]:
                for rel_pro in parameters[2]:
                    for matrix in parameters[1]:
                        for cross in parameters[0]:
                            # Running the artifact classifier with relative procent on each ICA interval
                            artifact_intervals = []
                            for intv_idx in range(intv_count):
                                folder_int = folder_path + "_time=" + str(time_start + time_intv[intv_idx]) + "-" + str(time_start +time_intv[intv_idx + 1])
                                artifact_intervals.append(ICA_subfunctions.artifact_classifier(folder_int, rel_pro, cross=cross, matrix=matrix))

                            # Running the broken electrode classifier with number of 10 s. intervals in row.
                            broken_pred = ICA_subfunctions.broken_channel_classifier(artifact_intervals, broken_interval_size = art_int)

                            # Output the predictions for the combination of parameters.
                            outputs.append([data[0], time_start, cross, matrix, rel_pro, art_int, artifact_intervals, broken_pred])
        
        # ends timer
        stop = timeit.default_timer() 
        total_time = (stop - start_cont)/60
        time_estimated = (stop - start)*len(data_set)/60
        print(str(total_time)[:4] + "min. Estimated time left with this speed: " + str(time_estimated - total_time)[:4] + "min.")
    return outputs

## Accuracies

$$
\text{ } \\
Accuracy = \frac{\text{Number of correct predictions}}{\text{Total number of predictions}} = \frac{TP + TN}{TP + TN + FP + FN}\\
$$

In [9]:
def binary_broken_electrode_accuracy(y_preds, y_labels):
    TP = 0 # True positive
    TN = 0 # True negative
    F = 0 # Incorrect - false
    accuracies = []
    
    for data_idx in range(len(y_preds)):
        if len(y_preds[data_idx]) == len(y_labels[data_idx]) == 0:
            # if both lists are empty. 0 labels and 0 predictions.
            TN += 1
            accuracies.append(1)
            
        elif len(y_preds[data_idx]) != len(y_labels[data_idx]):
            # if the lists are not same lenght. Not same number of predictions as number of labels
            F += 1
            accuracies.append(0)
        else:
            # check if predictions are correct.
            total_correct = 0
            for chn in range(len(y_preds[data_idx])):
                if y_preds[data_idx][chn] in y_labels[data_idx]:
                    total_correct += 1
            if total_correct == len(y_labels[data_idx]):
                TP += 1
                accuracies.append(1)
            else:
                F += 1
                accuracies.append(0)
    return (TP + TN) / (TP + TN + F), accuracies, {'TP':TP, 'TN':TN, 'F':F}

In [10]:
def broken_electrode_accuracy(y_preds, y_labels):
    TP = 0 #true_positive
    FP = 0 #false_positive
    TN = 0 #true_negative no broken channels predicted and none labelled.
    FN = 0 #false_negative
    accuracies = []
    
    for data_idx in range(len(y_preds)):
        # Running through each data point
        
        interval_accuracy = []
        for pred_chn in range(len(y_preds[data_idx])):
            # Go through each channel prediction
            
            if y_preds[data_idx][pred_chn] in y_labels[data_idx]:
                # Correctly predicts a broken channel.
                TP += 1
                interval_accuracy.append('TP')
            else:
                # Predict a broken channel, that was not broken.
                FP += 1
                interval_accuracy.append('FP')
        
        for label_chn in range(len(y_labels[data_idx])):
            if y_labels[data_idx][label_chn] not in y_preds[data_idx]:
                # Failed to predict a broken channel.
                FN += 1
                interval_accuracy.append('FN')
        
        if len(y_labels[data_idx]) == 0 and len(y_preds[data_idx]) == 0:
            # Predict no broken and no broken is correct.
            TN += 1
            interval_accuracy.append('TN')
        accuracies.append(interval_accuracy)
    return (TP + TN) / (TP + FP + FN + TN), accuracies, {'TP':TP, 'TN':TN, 'FP':FP, 'FN':FN}

In [11]:
def both_broken_electrode_accuracy(y_preds, y_labels, binary):
    if binary:
        acc_score, acc, types = binary_broken_electrode_accuracy(y_preds, y_labels)
    else:
        acc_score, acc, types = broken_electrode_accuracy(y_preds, y_labels)
    return acc_score, acc, types

## Result of the best combination on simulated test set.

In [12]:
#sim_break_test_prediction = optimise_set(sim_break_test_data, True, ['M_max', 'mm', 1.5, 2]) # calculation prediction of the best combination

In [13]:
sim_break_test_prediction = [['EEG_manual_100-25cha_0noise_1000', 10, 'M_max', 'mm', 1.5, 2, [[], [], ['F4'], [], [], ['ERA']], []], ['EEG_manual_100-25cha_1noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA']], ['ELA']], ['EEG_manual_100-25cha_2noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB'], ['ELA', 'ELB']], ['ELA', 'ELB']], ['EEG_manual_100-25cha_3noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB', 'ELC'], ['ELA', 'ELB', 'ELC'], ['ELA', 'ELB', 'ELC'], ['ELA', 'ELB', 'ELC'], ['ELA', 'ELB', 'ELC'], ['ELA', 'ELB', 'ELC']], ['ELA', 'ELB', 'ELC']], ['EEG_manual_100-25cha_4noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB', 'ELC', 'ELT'], ['ELA', 'ELB', 'ELC', 'ELT'], ['ELA', 'ELB', 'ELC', 'ELT'], ['ELA', 'ELB', 'ELC', 'ELT'], ['ELA', 'ELB', 'ELC', 'ELT'], ['ELA', 'ELB', 'ELC', 'ELT']], ['ELA', 'ELB', 'ELC', 'ELT']], ['EEG_manual_100-25cha_5noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE']], ['EEG_manual_100-25cha_6noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI']], ['EEG_manual_100-25cha_7noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA']], ['EEG_manual_100-25cha_8noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB']], ['EEG_manual_100-25cha_9noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC']], ['EEG_manual_100-25cha_10noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT']], ['EEG_manual_100-25cha_11noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE']], ['EEG_manual_100-25cha_12noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERI'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE', 'ERI'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT', 'ERE', 'ERI'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE', 'ERI'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE', 'ERI'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE', 'ERI']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERI', 'ERE']], ['EEG_manual_100-25cha_13noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE', 'ERI', 'EOGr'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE', 'ERI', 'EOGr'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE', 'ERI', 'EOGr'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERT', 'ERE', 'ERI', 'EOGr'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERI', 'EOGr'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE', 'ERI', 'EOGr']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE', 'ERI', 'EOGr']], ['EEG_manual_100-25cha_14noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE', 'ERI', 'EOGr', 'EOGl'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE', 'ERI', 'EOGr', 'EOGl'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERT', 'ERE', 'ERI', 'EOGr', 'EOGl'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE', 'ERI', 'EOGr'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERT', 'ERE', 'ERI', 'EOGr', 'EOGl'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE', 'ERI', 'EOGr', 'EOGl']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE', 'ERI', 'EOGr', 'EOGl']], ['EEG_manual_100-25cha_15noise_1000', 10, 'M_max', 'mm', 1.5, 2, [['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT', 'ERE', 'EOGl', 'EMGl'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERI', 'EOGr', 'EMGl'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERC', 'ERT', 'ERE', 'ERI', 'EOGr', 'EOGl'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl'], ['ELA', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERI', 'EOGl', 'EMGl']], ['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERB', 'ERC', 'ERT', 'ERE', 'EOGl', 'EMGl', 'ERA', 'ERI', 'EOGr']]]

In [14]:
def test_accuracies(outputs, data, binary):        
    y_pre_each_combination = [outputs[i][7] for i in range(0, len(outputs))]
    y_lab_each_combination = [data[j][3] for j in range(0, len(data))]
    acc_score, acc, types = both_broken_electrode_accuracy(y_pre_each_combination, y_lab_each_combination, binary)
    return acc_score, types
print(test_accuracies(sim_break_test_prediction, sim_break_test_data, binary=True))
print(test_accuracies(sim_break_test_prediction, sim_break_test_data, binary=False))

(1.0, {'TP': 15, 'TN': 1, 'F': 0})
(1.0, {'TP': 120, 'TN': 1, 'FP': 0, 'FN': 0})


In [15]:
for i in range(16):
    print(sim_break_test_prediction[i][7])
    print(sim_break_test_data[i][3])

[]
[]
['ELA']
['ELA']
['ELA', 'ELB']
['ELA', 'ELB']
['ELA', 'ELB', 'ELC']
['ELA', 'ELB', 'ELC']
['ELA', 'ELB', 'ELC', 'ELT']
['ELA', 'ELB', 'ELC', 'ELT']
['ELA', 'ELB', 'ELC', 'ELT', 'ELE']
['ELA', 'ELB', 'ELC', 'ELT', 'ELE']
['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI']
['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI']
['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA']
['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA']
['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB']
['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB']
['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC']
['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC']
['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT']
['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT']
['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE']
['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT', 'ERE']
['ELA', 'ELB', 'ELC', 'ELT', 'ELE', 'ELI', 'ERA'